In [ ]:
import sys
import json
import numpy as np
sys.path.insert(0, '..')
from src.embedding import Embedding, Collision, JointLimit
from src.forward_kinematics import ForwardKinematic
from src.dynamical_system import DynamicalSystem
from utils import franka_parameters
from utils.messenger import Client

In [ ]:
with open('../config/environment2.json') as file:
    config = json.load(file)

In [ ]:
# add an obstacle to the workspace
x = np.array(config['obstacles'])
# place an attractor in the configuration space
config_attractor = np.array([a * np.pi/180 for a in config['attractor']])

fk = ForwardKinematic(
    urdf_file=config['urdf'],
    gmm_configuration_file="../config/gmm_unit.json"
)

e = Embedding(embeddings=[Collision(x=x, fk=fk), JointLimit(limits=franka_parameters.joint_limits, booster=10)])

# define proportional and dissipation gains for the DS
k, d = 0.5, 1.5
ds = DynamicalSystem(stiffness=k*np.eye(fk.model.nq), dissipation=d*np.eye(fk.model.nq), attractor=config_attractor, embedding=e, dt=0.01)

In [ ]:
# initial conditions
q = np.array([a * np.pi/180 for a in config['initial_configuration']])
dq = np.array(config['initial_velocities'])

client = Client(port="5511")

In [ ]:
iterations = 10000
for _ in range(iterations):
    ddq = ds.compute_basic_switched_acceleration(q, dq, kappa=0.15)
    # the accelerations is sent to the simulation/robot
    client.send_request(ddq.squeeze().tolist())
    # the simulation/robot sends the real positions & veclocities back
    q, dq = np.split(client.get_reply(), 2)